<h1 align=center><font size = 5> Optimal Location for Opening a coffee shop in Toronto </font></h1>



the first thing to do is to import the needed library to do the analysis







In [1]:
# import necessary libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import requests
import json
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

%matplotlib inline
print('Packages installed  :)')

Packages installed  :)


In [2]:
link = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=954731857" 
wiki_page = requests.get(link)
wiki_data = pd.read_html(wiki_page.text) #reading the html page into a pandaframe 
wiki_data = wiki_data[0] #keeping only the first table we need 
df_read = wiki_data.groupby(['Postal code']).head() # Grouping by Postal Code
df_read= df_read.reset_index()
df_read.drop(['index'], axis = 'columns', inplace = True) # dropping the index column
df_read

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


dropping borough with missing value .

In [3]:
df_read= df_read[df_read.Borough != 'Not assigned']
df_read.head()


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [4]:
# groupby Postcode and combine Neighbourhood 
df_read= df_read.groupby(['Postal code','Borough']).Neighborhood.apply(lambda x: ','.join(x)).reset_index()
df_read.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
# cleaning the data more..... and getting the shape of the data
df_read['Neighborhood'][df_read['Neighborhood']=='Not assigned'] =df_read['Borough']
df_read.head()
df_read.shape

(103, 3)

getting the geospatial data 

In [6]:
!wget -q -O 'toronto_data.csv' https://cocl.us/Geospatial_data   # Download the csv file



In [7]:
geo_df=pd.read_csv('toronto_data.csv')
geo_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True) # renaming the postal code colum
geo_df.head()



,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now we combine the two dataframes  

In [31]:
df_toronto =df_read.join(geo_df.set_index('PostalCode'), on='Postal code')
df_toronto


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...,43.688905,-79.554724
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437


In [32]:
df_toronto.rename(columns={'Postal code': 'Postalcode'}, inplace=True) # renaming the postal code colum


Now everything looks clear and good , now we can explore and cluster the neighborhoods in Toronto
Before we get the data and start exploring it, let's download all the dependencies that we will need.


In [33]:
from geopy.geocoders import Nominatim


In [34]:

address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


printing the map of torronto and it's neihberhood

In [35]:
import folium
# Creating the map of Toronto
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# adding markers to map
for latitude, longitude, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

now we should define foursquare Credentials and version  

In [36]:
CLIENT_ID = 'WXPVAEVLCAS4NJHWTFHABI2UXJPRTVNXRUGXT32LBK53CLIS' # your Foursquare ID
CLIENT_SECRET = 'LWOOKRAGLZDBVWISPA02CENDPKC1XO13EKXQBM4AHUKELKMI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WXPVAEVLCAS4NJHWTFHABI2UXJPRTVNXRUGXT32LBK53CLIS
CLIENT_SECRET:LWOOKRAGLZDBVWISPA02CENDPKC1XO13EKXQBM4AHUKELKMI


In [39]:
postalcode_latitude= df_toronto.loc[4,'Latitude']# neighborhood latitude value
postalcode_longitude= df_toronto.loc[4,'Longitude']# neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(df_toronto.loc[4,'Postalcode'],postalcode_latitude, 
                                                               postalcode_longitude))

Latitude and longitude values of M1H are 43.773136, -79.23947609999999.


In [40]:
LIMIT=100
radius=500
venuss='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit{}'.format(CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    postalcode_latitude, 
    postalcode_longitude, 
    radius, 
    LIMIT)
venuss

'https://api.foursquare.com/v2/venues/explore?&client_id=WXPVAEVLCAS4NJHWTFHABI2UXJPRTVNXRUGXT32LBK53CLIS&client_secret=LWOOKRAGLZDBVWISPA02CENDPKC1XO13EKXQBM4AHUKELKMI&v=20180605&ll=43.773136,-79.23947609999999&radius=500&limit100'

now we examine what we have sending the url request

In [41]:
results = requests.get(venuss).json()
results

{'meta': {'code': 200, 'requestId': '60be07cc49d56e7e7e4bf69d'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b1711a6f964a520cbc123e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
          'suffix': '.png'},
         'id': '52af3ac83cf9994f4e043bf3',
         'name': 'Hakka Restaurant',
         'pluralName': 'Hakka Restaurants',
         'primary': True,
         'shortName': 'Hakka'}],
       'id': '4b1711a6f964a520cbc123e3',
       'location': {'address': '1920 Ellesmere Rd',
        'cc': 'CA',
        'city': 'Scarborough',
        'country': 'Canada',
        'crossStreet': 'at Bellamy Rd. N',
        'distance': 219,
        'formattedAddress': ['1920 Ellesmere Rd (at Bellamy Rd. N)',
         'Scarborough ON M1H 2V6',
         'Canada'],
    

now we should to get the diffrent categories of revenus in order to understand the data better 

In [42]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:

venuess = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venuess) # flatten JSON
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng'] #filter the columns
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head() # printing the first rows to get an overview of the data 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Federick Restaurant,Hakka Restaurant,43.774697,-79.241142
1,Drupati's Roti & Doubles,Caribbean Restaurant,43.775222,-79.241678
2,Thai One On,Thai Restaurant,43.774468,-79.241268
3,Centennial Recreation Centre,Athletics & Sports,43.774593,-79.236500
4,TD Canada Trust,Bank,43.774830,-79.241251


since we only intrested in toronto we should filter by the word "toronto" to get only the borough that intrest us 

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
toronto_data = df_toronto[df_toronto['Borough'].str.contains('Toronto')].reset_index(drop=True) #runing the function on the toronto data frame 
toronto_data.head()
#Run the function
Toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District / Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
Roselawn
Forest Hill North & West
The Annex / North Midtown / Yorkville
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Stn A PO Boxes
First Canadian Place / Underground city
Christie
Dufferin / Dovercourt Village
Little Portugal / Trinity
Brockton / Parkdale Village / Exhibition Place
High Park /

In [45]:
toronto_data.head

<bound method NDFrame.head of    Postalcode           Borough  ...   Latitude  Longitude
0         M4E      East Toronto  ...  43.676357 -79.293031
1         M4K      East Toronto  ...  43.679557 -79.352188
2         M4L      East Toronto  ...  43.668999 -79.315572
3         M4M      East Toronto  ...  43.659526 -79.340923
4         M4N   Central Toronto  ...  43.728020 -79.388790
5         M4P   Central Toronto  ...  43.712751 -79.390197
6         M4R   Central Toronto  ...  43.715383 -79.405678
7         M4S   Central Toronto  ...  43.704324 -79.388790
8         M4T   Central Toronto  ...  43.689574 -79.383160
9         M4V   Central Toronto  ...  43.686412 -79.400049
10        M4W  Downtown Toronto  ...  43.679563 -79.377529
11        M4X  Downtown Toronto  ...  43.667967 -79.367675
12        M4Y  Downtown Toronto  ...  43.665860 -79.383160
13        M5A  Downtown Toronto  ...  43.654260 -79.360636
14        M5B  Downtown Toronto  ...  43.657162 -79.378937
15        M5C  Downtown To

In [46]:
Toronto_venues.head() #printing the first 5 rows 


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Groseille's Grotto,43.674311,-79.289162,Playground
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth West / Riverdale,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [47]:
Toronto_venues.groupby('Neighborhood').count().head() #cheking how much revenus by neighborhood 


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,47,47,47,47,47,47
Brockton / Parkdale Village / Exhibition Place,21,21,21,21,21,21
Business reply mail Processing Centre,16,16,16,16,16,16
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,15,15,15,15,15,15
Central Bay Street,62,62,62,62,62,62


now we take a closer look at each neighborhood with somme analysis

In [48]:
onehotencod = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
onehotencod['Neighborhood'] = Toronto_venues['Neighborhood'] 
fixed_columns = [onehotencod.columns[-1]] + list(onehotencod.columns[:-1])
onehotencod = onehotencod[fixed_columns]
frame_grouped = onehotencod.groupby('Neighborhood').mean().reset_index()
frame_grouped.head() # printing the data to screen

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,...,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.021277,0.000000,0.0,0.0,0.0000,0.0,0.0,0.021277,0.063830,0.000000,0.000000,0.0,0.021277,0.021277,0.0,0.042553,0.0,0.0,0.000000,0.021277,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0000,...,0.000000,0.0,0.063830,0.0,0.000000,0.042553,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.021277,0.0,0.021277,0.0,0.0,0.000000,0.0,0.0,0.0,0.021277,0.000000,0.0,0.0,0.0,0.021277,0.0,0.0,0.0,0.0,0.0,0.0,0.042553,0.0,0.0,0.000000,0.0
1,Brockton / Parkdale Village / Exhibition Place,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.047619,0.000000,0.047619,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.095238,0.0000,...,0.000000,0.0,0.095238,0.0,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.047619,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,Business reply mail Processing Centre,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0625,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0625,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0625,0.0,0.0625,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.0,0.0,0.066667,0.066667,0.066667,0.066667,0.066667,0.133333,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.066667,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.066667,0.0,0.066667,0.0,0.000000,0.0000,...,0.000000,0.0,0.000000,0.0,0.066667,0.000000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,Central Bay Street,0.016129,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.016129,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.032258,0.016129,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.016129,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0000,...,0.032258,0.0,0.080645,0.0,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.064516,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.016129,0.0,0.0,0.016129,0.0


generating and printing the most revenus by order to get an idea about the best neighborhood in the area 

In [49]:
num_top_venues=7
def return_most_common_venues(row, num_top_venues):    
    row_cat = row.iloc[1:]
    row_cat_sorted = row_cat.sort_values(ascending=False)
    
    return row_cat_sorted.index.values[0:num_top_venues]

In [50]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = frame_grouped['Neighborhood']

for ind in np.arange(frame_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(frame_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Sandwich Place,Bakery,Seafood Restaurant,Vegetarian / Vegan Restaurant,Farmers Market,Beer Bar,Spa,Butcher
1,Brockton / Parkdale Village / Exhibition Place,Coffee Shop,Café,Breakfast Spot,Sandwich Place,Bakery,Italian Restaurant,Gym,Bar,Furniture / Home Store,Stadium
2,Business reply mail Processing Centre,Light Rail Station,Park,Pizza Place,Skate Park,Smoke Shop,Restaurant,Brewery,Burrito Place,Farmers Market,Fast Food Restaurant
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Terminal,Coffee Shop,Boat or Ferry,Boutique,Rental Car Location,Bar,Plane,Harbor / Marina,Sculpture Garden,Airport Service
4,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Japanese Restaurant,Café,Burger Joint,Salad Place,Restaurant,Bank


usinge the kmeans algorithm to cluster the neighboorhood 

In [58]:
# set number of clusters to 5 clusters
kclusters = 7

clustering_neighborhood = frame_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering_neighborhood)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood') #merging the data 
toronto_merged = toronto_merged[toronto_merged.Postalcode!= 'M9A'] 
toronto_merged = toronto_merged.astype({'Cluster Labels':int})
toronto_merged

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Playground,Pub,Wine Shop,Cosmetics Shop,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Grocery Store,Restaurant,Pub,Pizza Place,Lounge,Juice Bar
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572,1,Fast Food Restaurant,Park,Pizza Place,Pub,Steakhouse,Brewery,Board Shop,Italian Restaurant,Restaurant,Food & Drink Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Gastropub,Café,Bakery,Ice Cream Shop,Italian Restaurant,Latin American Restaurant,Convenience Store,Middle Eastern Restaurant,Stationery Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Business Service,Bus Line,Swim School,College Arts Building,Creperie,Dog Run,Distribution Center,Discount Store,Diner
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Hotel,Pizza Place,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Playground,Park,Gym / Fitness Center,College Gym
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Clothing Store,Coffee Shop,Ice Cream Shop,Chinese Restaurant,Pet Store,Café,Diner,Spa,Sporting Goods Shop,Fast Food Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Pizza Place,Gym,Dessert Shop,Coffee Shop,Sandwich Place,Sushi Restaurant,Greek Restaurant,Seafood Restaurant,Indian Restaurant,Italian Restaurant
8,M4T,Central Toronto,Moore Park / Summerhill East,43.689574,-79.383160,2,Restaurant,Wine Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio
9,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,43.686412,-79.400049,1,Coffee Shop,Athletics & Sports,Sushi Restaurant,Bagel Shop,American Restaurant,Bank,Pizza Place,Light Rail Station,Vietnamese Restaurant,Diner


creating a map to better visualuaze the data 

In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)    
map_clusters

printing the 5 cluster to see what's the best place to open a coffee shop 

In [66]:
cluster_0=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_0



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Playground,Pub,Wine Shop,Cosmetics Shop,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


In [89]:
cluster_1=toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_1

,Borough,Cluster Labels,firstcommonvenue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Grocery Store,Restaurant,Pub,Pizza Place,Lounge,Juice Bar
2,East Toronto,1,Fast Food Restaurant,Park,Pizza Place,Pub,Steakhouse,Brewery,Board Shop,Italian Restaurant,Restaurant,Food & Drink Shop
3,East Toronto,1,Coffee Shop,Gastropub,Café,Bakery,Ice Cream Shop,Italian Restaurant,Latin American Restaurant,Convenience Store,Middle Eastern Restaurant,Stationery Store
5,Central Toronto,1,Hotel,Pizza Place,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Playground,Park,Gym / Fitness Center,College Gym
6,Central Toronto,1,Clothing Store,Coffee Shop,Ice Cream Shop,Chinese Restaurant,Pet Store,Café,Diner,Spa,Sporting Goods Shop,Fast Food Restaurant
7,Central Toronto,1,Pizza Place,Gym,Dessert Shop,Coffee Shop,Sandwich Place,Sushi Restaurant,Greek Restaurant,Seafood Restaurant,Indian Restaurant,Italian Restaurant
9,Central Toronto,1,Coffee Shop,Athletics & Sports,Sushi Restaurant,Bagel Shop,American Restaurant,Bank,Pizza Place,Light Rail Station,Vietnamese Restaurant,Diner
11,Downtown Toronto,1,Coffee Shop,Café,Pizza Place,Bakery,Pub,Restaurant,Italian Restaurant,Sandwich Place,Park,General Entertainment
12,Downtown Toronto,1,Sushi Restaurant,Japanese Restaurant,Restaurant,Coffee Shop,Gym,Fast Food Restaurant,Indian Restaurant,Burrito Place,Mediterranean Restaurant,Gay Bar
13,Downtown Toronto,1,Coffee Shop,Bakery,Park,Pub,Café,Greek Restaurant,Event Space,Bank,Farmers Market,Spa


In [90]:
cluster_2=toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_2

,Borough,Cluster Labels,firstcommonvenue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,2,Restaurant,Wine Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio


In [91]:
cluster_3=toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,firstcommonvenue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,4,Park,Business Service,Bus Line,Swim School,College Arts Building,Creperie,Dog Run,Distribution Center,Discount Store,Diner


In [93]:
cluster_4=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


In [ ]:
total_coffeeshop = frame_grouped['Coffee Shop']+frame_grouped['Café']
total_coffeeshop
#df_cofee = pd.DataFrame(data={'Neighborhood': frame_grouped['Neighborhood'],'Totalcoffeeshop':total_coffeeshop})
#df_cofee.head(10)

0     0.118644
1     0.217391
2     0.000000
3     0.000000
4     0.245902
5     0.250000
6     0.088608
7     0.200000
8     0.117647
9     0.000000
10    0.066667
11    0.180000
12    0.000000
13    0.120000
14    0.170000
15    0.076923
16    0.000000
17    0.147541
18    0.000000
19    0.116279
20    0.000000
21    0.150000
22    0.071429
23    0.235294
24    0.227273
25    0.164835
26    0.000000
27    0.000000
28    0.147059
29    0.126582
30    0.130435
31    0.160000
32    0.135135
33    0.142857
34    0.263158
35    0.166667
36    0.121951
37    0.180000
38    0.187500
dtype: float64

Analyzing the results we can clearly see that the best neighborhood to open a coffee shop is located in the first cluster ( est toronto and tornto center and less better but still good is downtown toronto ) and by that we can conclud our analysis , knowing confidently that we are in the best neigboorhood to start our coffee busniness.  